# 🚀 MITRE ATT&CK TTP Mapping - Colab Eğitim

Google Colab'da GPU ile hızlı eğitim için hazırlanmış notebook.

In [1]:
# GPU kontrolü
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️  GPU yok - CPU kullanılacak (yavaş olabilir)")

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4
CUDA version: 12.6
Memory: 14.74 GB


## 1. Kurulum

Gerekli kütüphaneleri yükle:

In [2]:
# Gerekli kütüphaneleri yükle
!pip install -q transformers datasets torch scikit-learn tqdm

print("✓ Kütüphaneler yüklendi!")

✓ Kütüphaneler yüklendi!


## 2. Eğitim Başlat 🚀

Aşağıdaki hücrelerden birini çalıştır. Sonuçlar `outputs/` klasörüne kaydedilir:

In [ ]:
# Proje kurulumu
import os
import sys

print("📥 Proje hazırlanıyor...")

# GitHub'dan projeyi klonla
if not os.path.exists('/content/Mitre_Attack_TTP_Mapping'):
    !git clone https://github.com/Aliekinozcetin/Mitre_Attack_TTP_Mapping.git
else:
    print("✅ Proje zaten mevcut")

# Proje dizinine geç
os.chdir('/content/Mitre_Attack_TTP_Mapping')
sys.path.insert(0, '/content/Mitre_Attack_TTP_Mapping')

print(f"✅ Hazır! Eğitim başlatılabilir.")

📍 Başlangıç dizini: /content
🔧 Google Colab tespit edildi!
✅ Proje zaten mevcut
✅ Proje dizinine geçildi: /content/Mitre_Attack_TTP_Mapping
✅ src/ klasörü bulundu
📂 İçerik: main.py, src, .gitattributes, COLAB_GUIDE.md, run_training.ipynb, outputs, requirements.txt, CTI_MITRE_ATTACK_DATASETS.md
🎯 Hazır! Eğitime başlayabilirsin.


In [ ]:
# BERT-base ile eğitim
from src.data_loader import prepare_data
from src.model import load_model
from src.train import train_model
from src.evaluate import evaluate_model
import torch
import json
from datetime import datetime
import os

print("🚀 Eğitim başlıyor...")

# Konfigürasyon
model_name = "bert-base-uncased"
epochs = 3
batch_size = 32
learning_rate = 2e-5
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Model: {model_name}")
print(f"Device: {device}")
print(f"Epochs: {epochs}, Batch: {batch_size}\n")

# Veri hazırla
print("📊 Veri yükleniyor...")
data = prepare_data(model_name=model_name, max_length=512)

# Model yükle
print("🤖 Model yükleniyor...")
model = load_model(model_name=model_name, num_labels=data['num_labels'], device=device)

# Eğit
print("💪 Eğitim başladı...")
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"./outputs/{model_name.replace('/', '_')}_{timestamp}"

history = train_model(
    model=model,
    train_dataset=data['train_dataset'],
    test_dataset=data['test_dataset'],
    output_dir=output_dir,
    batch_size=batch_size,
    learning_rate=learning_rate,
    num_epochs=epochs,
    warmup_steps=500,
    device=device
)

# Değerlendir
print("📈 Model değerlendiriliyor...")
metrics = evaluate_model(
    model=model,
    test_dataset=data['test_dataset'],
    batch_size=batch_size,
    device=device,
    threshold=0.5,
    label_list=data['label_list']
)

# Sonuçları kaydet
os.makedirs(output_dir, exist_ok=True)

with open(f"{output_dir}/labels.json", 'w') as f:
    json.dump(data['label_list'], f, indent=2)

with open(f"{output_dir}/training_history.json", 'w') as f:
    json.dump(history, f, indent=2)

metrics_to_save = {k: float(v) if isinstance(v, (float, int)) else v 
                   for k, v in metrics.items() 
                   if k not in ['predictions', 'labels']}
with open(f"{output_dir}/evaluation_metrics.json", 'w') as f:
    json.dump(metrics_to_save, f, indent=2)

print(f"\n✅ Eğitim tamamlandı!")
print(f"📁 Sonuçlar: {output_dir}")
print(f"📈 Micro F1: {metrics['micro_f1']:.4f}")

🚀 Eğitim başlıyor...
Model: bert-base-uncased
Device: cuda
Epochs: 3, Batch: 32

📊 Veri yükleniyor...
Loading dataset: tumeteor/Security-TTP-Mapping
Available splits: ['train', 'validation', 'test']
Available splits: ['train', 'validation', 'test']
Using validation split as test set
Train samples: 14936
Test samples: 2630
Total unique labels (MITRE ATT&CK Techniques): 499
Using validation split as test set
Train samples: 14936
Test samples: 2630
Total unique labels (MITRE ATT&CK Techniques): 499
Loading tokenizer: bert-base-uncased
Loading tokenizer: bert-base-uncased
Tokenizing texts...
Tokenizing texts...
Data preparation complete!
🤖 Model yükleniyor...
Loading model: bert-base-uncased
Number of labels: 499
Data preparation complete!
🤖 Model yükleniyor...
Loading model: bert-base-uncased
Number of labels: 499


OutOfMemoryError: CUDA out of memory. Tried to allocate 90.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 64.12 MiB is free. Process 677541 has 14.68 GiB memory in use. Of the allocated memory 14.52 GiB is allocated by PyTorch, and 33.02 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Sonuçları ZIP olarak indir
import os
from datetime import datetime

if os.path.exists('./outputs') and os.listdir('./outputs'):
    print("📦 ZIP hazırlanıyor...")
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    zip_name = f"training_results_{timestamp}.zip"
    
    !zip -r {zip_name} outputs/
    
    from google.colab import files
    files.download(zip_name)
    
    print(f"✅ {zip_name} indirildi!")
else:
    print("❌ Henüz eğitim sonucu yok - önce eğitimi çalıştır!")

## 3. Sonuçları İndir 📥

Eğitim bitince sonuçları ZIP olarak indir:

In [ ]:
# SecBERT (security domain) ile eğitim
from src.data_loader import prepare_data
from src.model import load_model
from src.train import train_model
from src.evaluate import evaluate_model
import torch
import json
from datetime import datetime
import os

print("🚀 Eğitim başlıyor...")

# Konfigürasyon
model_name = "jackaduma/SecBERT"
epochs = 3
batch_size = 16
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Model: {model_name}")
print(f"Device: {device}")
print(f"Epochs: {epochs}, Batch: {batch_size}\n")

# Veri hazırla
print("📊 Veri yükleniyor...")
data = prepare_data(model_name=model_name, max_length=512)

# Model yükle
print("🤖 Model yükleniyor...")
model = load_model(model_name=model_name, num_labels=data['num_labels'], device=device)

# Eğit
print("💪 Eğitim başladı...")
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"./outputs/{model_name.replace('/', '_')}_{timestamp}"

history = train_model(
    model=model,
    train_dataset=data['train_dataset'],
    test_dataset=data['test_dataset'],
    output_dir=output_dir,
    batch_size=batch_size,
    learning_rate=2e-5,
    num_epochs=epochs,
    warmup_steps=500,
    device=device
)

# Değerlendir
print("📈 Model değerlendiriliyor...")
metrics = evaluate_model(
    model=model,
    test_dataset=data['test_dataset'],
    batch_size=batch_size,
    device=device,
    threshold=0.5,
    label_list=data['label_list']
)

# Sonuçları kaydet
os.makedirs(output_dir, exist_ok=True)

with open(f"{output_dir}/labels.json", 'w') as f:
    json.dump(data['label_list'], f, indent=2)

with open(f"{output_dir}/training_history.json", 'w') as f:
    json.dump(history, f, indent=2)

metrics_to_save = {k: float(v) if isinstance(v, (float, int)) else v 
                   for k, v in metrics.items() 
                   if k not in ['predictions', 'labels']}
with open(f"{output_dir}/evaluation_metrics.json", 'w') as f:
    json.dump(metrics_to_save, f, indent=2)

print(f"\n✅ Eğitim tamamlandı!")
print(f"📁 Sonuçlar: {output_dir}")
print(f"📈 Micro F1: {metrics['micro_f1']:.4f}")

In [ ]:
# Sonuçları görüntüle
import json
import os

output_path = './outputs'
if os.path.exists(output_path):
    output_dirs = [d for d in os.listdir(output_path) if os.path.isdir(os.path.join(output_path, d))]
    
    if output_dirs:
        latest_dir = sorted(output_dirs)[-1]
        print(f"📁 En son eğitim: {latest_dir}\n")
        
        metrics_file = f"{output_path}/{latest_dir}/evaluation_metrics.json"
        if os.path.exists(metrics_file):
            with open(metrics_file) as f:
                metrics = json.load(f)
            
            print("📈 Değerlendirme Metrikleri:")
            print(f"  Micro F1:    {metrics.get('micro_f1', 0):.4f}")
            print(f"  Macro F1:    {metrics.get('macro_f1', 0):.4f}")
            print(f"  Samples F1:  {metrics.get('samples_f1', 0):.4f}")
            print(f"  Precision:   {metrics.get('micro_precision', 0):.4f}")
            print(f"  Recall:      {metrics.get('micro_recall', 0):.4f}")
        
        history_file = f"{output_path}/{latest_dir}/training_history.json"
        if os.path.exists(history_file):
            with open(history_file) as f:
                history = json.load(f)
            print(f"\n📉 Final Loss: {history['train_loss'][-1]:.4f}")
    else:
        print("⚠️  Henüz eğitim sonucu yok")
else:
    print("⚠️  outputs/ klasörü bulunamadı")